In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

from src import EnergyTrading, GroupDistributor

In [23]:
# file_path = "data/meter_month/apt_780_201809.csv"
file_path = "data/meter_month/apt_1_month_9.csv"

trade = EnergyTrading(file_path, month=9)
dist = GroupDistributor(file_path).set_init_csv(month=9)

households_kwh = dist.meter_month['usage (kWh)'].sum()
public_kwh = households_kwh * 0.25
apt = households_kwh + public_kwh

In [24]:
trade.datas.head()

,name,usage (kWh)
0,아파트1-104-1206,172
1,아파트1-104-303,6
2,아파트1-104-1307,98
3,아파트1-104-1208,6
4,아파트1-104-408,41


In [25]:
dist.meter_month

,name,usage (kWh)
0,아파트1-104-1206,172
1,아파트1-104-303,6
2,아파트1-104-1307,98
3,아파트1-104-1208,6
4,아파트1-104-408,41
...,...,...
103,아파트1-103-903,277
104,아파트1-103-1106,179
105,아파트1-103-705,192
106,아파트1-103-1505,314


### EneryTrading

In [47]:
trade.set_init()
trade.simulate()
trade.set_settlement_infos()
sim_df = trade.simulation_datas

288
36795.2
[16957.6  1469.8 10053.4  1469.8  4735.3 25897.  22702.7 27588.1 17890.6
 18916.9 32097.7 25521.2 19196.8 45626.5 29655.  51075.6 20823.7 30970.3
 25897.  30218.7 20447.9 19196.8 17330.8 30970.3 21387.4 18450.4  9400.3
 45626.5 37922.6 26836.5 52015.1 33976.7 15184.9 19383.4 27963.9  3895.6
 29655.  28151.8 41304.8 22890.6 24581.7 29467.1 20447.9 69432.6 29467.1
 31721.9 71958.  26084.9 37358.9 14718.4 38862.1 14811.7 39050.  23454.3
 36419.4 20635.8 15838.  21199.5 29279.2 51827.2 35479.9 21575.3 70835.6
 26460.7 18170.5 42244.3 16211.2  8374.  13878.7  9866.8 14158.6 12199.3
 25521.2 14905.    910.   1563.1 15744.7 17330.8 15278.2 17517.4 54082.
 36043.6 22514.8 18730.3 44123.3 30406.6 75886.4 35479.9 53142.5 47129.7
 17144.2 27588.1 31721.9 38674.2 45438.6 28527.6 49948.2 52390.9 15744.7
 33976.7 30218.7 26272.8 50324.  34728.3 17610.7 18823.6 41680.6 25709.1]


In [48]:
trade.datas.head(10)

,name,usage (kWh),전력량요금,공용부요금
0,아파트1-104-1206,172,16957.6,8742.811111
1,아파트1-104-303,6,1469.8,8742.811111
2,아파트1-104-1307,98,10053.4,8742.811111
3,아파트1-104-1208,6,1469.8,8742.811111
4,아파트1-104-408,41,4735.3,8742.811111
5,아파트1-104-203,230,25897.0,8742.811111
6,아파트1-103-1402,213,22702.7,8742.811111
7,아파트1-103-402,239,27588.1,8742.811111
8,아파트1-103-1201,182,17890.6,8742.811111
9,아파트1-103-801,193,18916.9,8742.811111


In [49]:
sim_df.sort_values(by=["차이"], ascending=False).head(10)

,name,usage (kWh),전력량 요금,거래 전력량요금(거래가격 적용 X),공용부요금(거래 적용 X),거래 전력량요금(거래 적용 O),공용부요금(거래 적용 O),차이
86,아파트1-101-1302,259.0,75886.4,29746.1,12496.460185,56863.731545,12496.460185,19022.668455
46,아파트1-103-1805,260.0,71958.0,29934.0,12496.460185,54681.885336,12496.460185,17276.114664
62,아파트1-103-703,261.0,70835.6,30121.9,12496.460185,54104.741437,12496.460185,16730.858563
43,아파트1-102-802,261.0,69432.6,30121.9,12496.460185,53332.516437,12496.460185,16100.083563
74,아파트1-104-1006,200.0,910.0,18660.0,12496.460185,-10182.353378,12496.460185,11092.353378
3,아파트1-104-1208,200.0,1469.8,18660.0,12496.460185,-9317.082777,12496.460185,10786.882777
1,아파트1-104-303,200.0,1469.8,18660.0,12496.460185,-9317.082777,12496.460185,10786.882777
75,아파트1-104-902,200.0,1563.1,18660.0,12496.460185,-9172.87101,12496.460185,10735.97101
35,아파트1-101-704,200.0,3895.6,18660.0,12496.460185,-5567.576837,12496.460185,9463.176837
4,아파트1-104-408,200.0,4735.3,18660.0,12496.460185,-4269.670935,12496.460185,9004.970935


### Distributor

In [37]:
dist.set_calc(kwh=apt)
dist.set_cont()
dist_df = dist.distribute_table()

dist_df.drop_duplicates('기여도').sort_values(['기여도'])

,가구명,공용부요금,기여도,기여도 기본요금,기여도 전력량요금,나머지 요금,기여도 요금,오차
1,아파트1-104-303,8743,0.020408,1.0,178.0,7457.444444,7636.0,1107.0
2,아파트1-104-1307,8743,0.061224,2.0,534.0,7457.444444,7993.0,750.0
0,아파트1-104-1206,8743,0.102041,3.0,890.0,7457.444444,8350.0,393.0
5,아파트1-104-203,8743,0.142857,4.0,1245.0,7457.444444,8706.0,37.0
10,아파트1-103-502,8743,0.183673,5.0,1601.0,7457.444444,9063.0,-320.0
13,아파트1-103-1502,8743,0.22449,6.0,1957.0,7457.444444,9420.0,-677.0
43,아파트1-102-802,8743,0.265306,7.0,2313.0,7457.444444,9777.0,-1034.0


In [40]:
dist.calc.apt_obj.steps

array([200.,  88.,   0.])

### Distributor in min-max-normalization

In [7]:
from src.utils import min_max_normalization

In [8]:
dist.set_calc(kwh=apt)
dist.set_cont(norm=min_max_normalization)
dist_df = dist.distribute_table()

dist_df.drop_duplicates('기여도').sort_values(['기여도'])

,가구명,공용부요금,기여도,기여도 기본요금,기여도 전력량요금,나머지 요금,기여도 요금,오차
1,아파트1-104-303,8743,0.0,0.0,0.0,4223.342593,4223.0,4520.0
2,아파트1-104-1307,8743,0.166667,4.0,1453.0,4223.342593,5680.0,3063.0
0,아파트1-104-1206,8743,0.333333,8.0,2906.0,4223.342593,7137.0,1606.0
5,아파트1-104-203,8743,0.5,12.0,4359.0,4223.342593,8594.0,149.0
10,아파트1-103-502,8743,0.666667,17.0,5812.0,4223.342593,10052.0,-1309.0
13,아파트1-103-1502,8743,0.833333,21.0,7265.0,4223.342593,11509.0,-2766.0
43,아파트1-102-802,8743,1.0,25.0,8718.0,4223.342593,12966.0,-4223.0
